In [1]:
import numpy as np
import openslide
import cv2
import PIL.Image as Image
import matplotlib.pyplot as plt
import tqdm
from skimage import transform
import os

In [2]:
from modules.wsiTile import WSITile
from modules.wsiRegister import WSIRegister
from modules.utility import slide_summary
import config
%load_ext autoreload
%autoreload 2

## Load WSI 

In [3]:
slideHE = openslide.open_slide(config.TARGET_SLIDE_PATH)
slideGI = openslide.open_slide(config.SOURCE_SLIDE_PATH)



WSI information

In [4]:
print("HE")
_, objectiveHE, dimsHE, factorsHE = slide_summary(slideHE, verbose=True)
print("\nGIESMA")
_, objectiveGI, dimsGI, factorsGI = slide_summary(slideGI, verbose=True)

HE
the objective power is 40
Dimensions are: ((61440, 73728), (30720, 36864), (15360, 18432), (7680, 9216), (3840, 4608), (1920, 2304), (960, 1152), (480, 576), (240, 288))
Factors are: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0)

GIESMA
the objective power is 40
Dimensions are: ((76800, 59904), (38400, 29952), (19200, 14976), (9600, 7488), (4800, 3744), (2400, 1872), (1200, 936), (600, 468), (300, 234))
Factors are: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0)


In [ ]:
slide_thumb_HE = slideHE.get_thumbnail(size=(800, 800))
print(np.asarray(slide_thumb_HE).shape)
slide_thumb_HE

In [ ]:
slide_thumb_GI = slideGI.get_thumbnail(size=(800, 800))
slide_thumb_GI

In [ ]:
shape = (dimsHE[6][1], dimsHE[6][0])
imgHE = WSITile(slideHE, level=6, top_left_coord=(0, 0), shape=shape)
shape = (dimsGI[6][1], dimsGI[6][0])
imgGI = WSITile(slideGI, level=6, top_left_coord=(0, 0), shape=shape)

ax = 'off'

# Plot two images side by side
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(imgHE.tile)
ax[0].set_title('HE')
ax[0].axis('off')
ax[1].imshow(imgGI.tile)
ax[1].set_title('Giesma')
ax[1].axis('off')
plt.show()

## Create an object WSI_alinger

In [47]:
wsi_reg = WSIRegister(target_slide=slideHE,
                      source_slide=slideGI)

## Stain Deconvolution
extract cytoplasm

In [ ]:
plt.imshow(imgHE.cyto, cmap='gray')
plt.title('Eosin HE')
plt.show()

plt.imshow(imgGI.cyto, cmap='gray')
plt.title('Eosin Giesma')
plt.show()

## SIFT + RANSAC

In [ ]:
p_points, q_points = wsi_reg.find_matches_with_SIFT_and_RANSAC(target=imgHE.cyto,
                                                               source=imgGI.cyto,
                                                               min_match_count=12,
                                                               verbose=True,
                                                               plot=True,
                                                               target_to_show=imgHE.tile,
                                                               source_to_show=imgGI.tile,
                                                               plot_title='SIFT detection + RANSAC')



p_points, q_points = find_matches(imgHE.cyto, 
                                  imgGI.cyto,
                                  min_match_count=10,
                                  verbose=True,
                                  plot=True,
                                  target_to_show=imgHE.tile,
                                  source_to_show=imgGI.tile,
                                  plot_title='SIFT detection + RANSAC')

## Transformation Parameters: S, R, T

In [15]:
S, R, T = wsi_reg.evaluate_transformation_param(p=p_points,
                                                q=q_points,
                                                verbose=True)

Shape covariance matrix is (2, 2)
det(R) = -0.9999999999999998
S = 3.02778767479297
R = [[-0.62460845  0.78093808]
 [ 0.78093808  0.62460845]]
T = [  70.90183822 -420.87420441]


In [ ]:
wsi_reg.plot_aligned_points(p=p_points,
                            q=q_points,
                            S=S,
                            R=R,
                            T=T)

In [ ]:
source_reg = wsi_reg.image_registration(target=imgHE.tile_grayscale, 
                                        source=imgGI.tile_grayscale,
                                        S=S,
                                        R=R,
                                        T=T)

In [ ]:
merged_img = wsi_reg.merge_registered_imgs(target=imgHE.tile_grayscale,
                                   source_reg=source_reg)

plt.imshow(merged_img)
plt.axis('off')
plt.show()

## Pyramidal alignment

In [37]:
REF_LEVEL = 6 # level where the gloab alignment is performed
LEVEL = [5, 4, 3, 2, 1, 0]
coords_first_inner_level = np.asarray([150, 140])
TOP_LEFT = [coords_first_inner_level * 2**i for i in [0, 1, 2, 3, 4, 5]]
#TOP_LEFT = [(70, 100), (140, 200), (3000, 3000), (6000, 6000), (12000, 12000), (24000, 24000)]
DIMS = (256, 256)

In [ ]:
reg_img5 = wsi_reg.pyramid_alignment(level=LEVEL[0], 
                                     dims=DIMS, 
                                     top_left=TOP_LEFT[0], 
                                     R=R, 
                                     S=S, 
                                     T=T, 
                                     ref_level=REF_LEVEL, 
                                     plot=True)

In [ ]:
reg_img4 = wsi_reg.pyramid_alignment(level=LEVEL[1], 
                                     dims=DIMS, 
                                     top_left=TOP_LEFT[1], 
                                     R=R, 
                                     S=S, 
                                     T=T, 
                                     ref_level=REF_LEVEL, 
                                     plot=True)

In [ ]:
reg_img3 = wsi_reg.pyramid_alignment(level=LEVEL[2], 
                                     dims=DIMS, 
                                     top_left=TOP_LEFT[2], 
                                     R=R, 
                                     S=S, 
                                     T=T, 
                                     ref_level=REF_LEVEL, 
                                     plot=True)

In [ ]:
reg_img2 = wsi_reg.pyramid_alignment(level=LEVEL[3], 
                                     dims=DIMS, 
                                     top_left=TOP_LEFT[3], 
                                     R=R, 
                                     S=S, 
                                     T=T, 
                                     ref_level=REF_LEVEL, 
                                     plot=True)

In [ ]:
reg_img1 = wsi_reg.pyramid_alignment(level=LEVEL[4], 
                                     dims=DIMS, 
                                     top_left=TOP_LEFT[4], 
                                     R=R, 
                                     S=S, 
                                     T=T, 
                                     ref_level=REF_LEVEL, 
                                     plot=True)

In [ ]:
reg_img0 = wsi_reg.pyramid_alignment(level=LEVEL[5], 
                                     dims=DIMS, 
                                     top_left=TOP_LEFT[5], 
                                     R=R, 
                                     S=S, 
                                     T=T, 
                                     ref_level=REF_LEVEL, 
                                     plot=True)

## Local iterative registration

Select top-left corners to cover the great majority of the image

In [ ]:
plt.imshow(imgHE.tile)
x = np.arange(80, 450, 50)
y = [80 for _ in range(len(x))]
plt.scatter(x, y, s=8, c='blue')
plt.axis('off')
plt.show()

In [17]:
LEVEL_START = 6
LEVEL_STOP = 2
TILE_SIZE = (256, 256)

For each selected top-left corner, perform both static and iterative registration

In [18]:
save_path = os.path.join(config.SAVE_FOLDER, 'Static_VS_Iterative')
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:

for idx in tqdm.tqdm(range(len(y))):
    
    # Evaluate the transformation parameters R, S and T for each level
    top_left = (int(y[idx]), int(x[idx]))
    parameters = wsi_reg.local_iterative_registration(level_start=LEVEL_START, 
                                                      level_stop=LEVEL_STOP, 
                                                      top_left_start=top_left, 
                                                      tile_size=TILE_SIZE,
                                                      min_match_count=4,
                                                      plot=False)
    
    # Use the LEVEL START parameters for the static pyramidal registration
    S_ref, R_ref, T_ref = parameters[f'level_{LEVEL_START}']
    static_reg_imgs = wsi_reg.stack_static_registration(ref_level=LEVEL_START, 
                                                        top_left_start=top_left, 
                                                        tile_size=TILE_SIZE, 
                                                        S=S_ref, 
                                                        R=R_ref, 
                                                        T=T_ref)
    
    # Perform iterative pyramidal registration
    iter_reg_imgs = wsi_reg.stack_iterative_registration(ref_level=LEVEL_START, 
                                                         top_left_start=top_left, 
                                                         tile_size=TILE_SIZE, 
                                                         parameters=parameters)
    
    # Save images 
    levels = np.arange(LEVEL_START-1, -1, -1) # from level start to zero
    for i, level in enumerate(levels):
        path = os.path.join(save_path, f'level_{level}') 
        if not os.path.exists(path):
            os.makedirs(path)
        plt.imsave(os.path.join(path, f'static_{idx}.png'), static_reg_imgs[i])
        plt.imsave(os.path.join(path, f'iterative_{idx}.png'), iter_reg_imgs[i])
        
    

## Compare Static vs Iterative with Mutual Image Information

In [20]:
LEVEL = [0, 1, 2, 3, 4, 5]

static_mean_mii = []
iter_mean_mii = []
static_std_mii = []
iter_std_mii = []

for level in LEVEL:
    path = os.path.join(save_path, f'level_{level}')
    num = int(len(os.listdir(path)) / 2)
    imgs_iter = []
    imgs_static = []
    for i in range(num):
        filename_iter = os.path.join(path, f'iterative_{i}.png')
        filename_static = os.path.join(path, f'static_{i}.png')
        img_iter = Image.open(filename_iter)
        img_static = Image.open(filename_static)
        imgs_iter.append(np.asarray(img_iter))
        imgs_static.append(np.asarray(img_static))
    mii_1 = []
    mii_2 = []
    for i in range(len(imgs_iter)):
        mii_1.append(wsi_reg.mii_evaluation(imgs_iter[i]))
        mii_2.append(wsi_reg.mii_evaluation(imgs_static[i]))
    
    iter_mean_mii.append(np.mean(mii_1))
    static_mean_mii.append(np.mean(mii_2))
    iter_std_mii.append(np.std(mii_1))
    static_std_mii.append(np.std(mii_2))


In [ ]:
# Scatter plot with y-errors
plt.errorbar(LEVEL, 
             static_mean_mii, 
             yerr=static_std_mii, 
             fmt='o', 
             capsize=5, markersize=5, color='blue', ecolor='blue', label='Static method', elinewidth=0.5)

plt.errorbar(LEVEL, 
             iter_mean_mii, 
             yerr=static_std_mii, 
             fmt='o', 
             capsize=5, markersize=5, color='red', ecolor='red', label='Iterative method', elinewidth=0.5)

# Aggiungi etichette e titolo
plt.xlabel('level')
plt.ylabel('MII')
plt.grid()
plt.title('Mutual Image Information')

# Aggiungi legenda
plt.legend()

# Mostra il plot
plt.show()

In [ ]:
for level in tqdm.tqdm(LEVEL):
    for idx in range(num):
        img1 = Image.open(os.path.join(save_path, f'level_{level}/iterative_{idx}.png'))
        img2 = Image.open(os.path.join(save_path, f'level_{level}/static_{idx}.png'))

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

        ax1.imshow(img1)
        ax1.set_title(f'Iterative method-Level {level}')
        ax1.axis('off')

        ax2.imshow(img2)
        ax2.set_title(f'Static method-Level {level}')
        ax2.axis('off')
        
        plt.show()
        
        